In [ ]:
from ncbi_genome import NCBIGenome
from covid19_genome import Covid19Genome

In [ ]:
ncbi_genome = NCBIGenome()

## Extract the taxa names from the file
with open("pathogenic-bacteria-taxa.txt", "r") as f:
    taxa = [line.strip() for line in f]

## Download the accessions
ncbi_genome.downloadAccessions(taxa, 100)

In [ ]:
covid19_genome = Covid19Genome()
covid19_genome.downloadLineages(accs_thresh = 256, max_accs = 256)